In [1]:
import pandas as pd
import re
from datetime import datetime
import numpy as np
pd.options.mode.chained_assignment = None  # |default='warn'

This algorithm scores **0.99441** on Shopee Code League - Logistics competition.

In [2]:
df_sla = pd.read_excel("dataset/SLA_matrix.xlsx")

In [3]:
## SLA Matrix
df_sla

,1st Attempt SLA (Working Days),Unnamed: 1,Destination (Buyer),Unnamed: 3,Unnamed: 4,Unnamed: 5
0,NaN,NaN,Metro Manila,Luzon,Visayas,Mindanao
1,Origin\n(Seller),Metro Manila,3 working days,5 working days,7 working days,7 working days
2,NaN,Luzon,5 working days,5 working days,7 working days,7 working days
3,NaN,Visayas,7 working days,7 working days,7 working days,7 working days
4,NaN,Mindanao,7 working days,7 working days,7 working days,7 working days
5,NaN,NaN,NaN,NaN,NaN,NaN
6,"Working Days are defined as Mon - Sat, Excludi...",NaN,NaN,NaN,NaN,NaN
7,SLA calculation begins from the next day after...,NaN,NaN,NaN,NaN,NaN
8,2nd Attempt must be no later than 3 working da...,NaN,NaN,NaN,NaN,NaN


In [4]:
## Delivery data
df_delivery = pd.read_csv("dataset/delivery_orders_march.csv")

In [5]:
df_delivery.shape

(3176313, 6)

In [6]:
df_delivery.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,1583138397,1.583385e+09,NaN,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...
1,2219624609,1583309968,1.583463e+09,1.583799e+09,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ..."
2,2220979489,1583306434,1.583460e+09,NaN,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M..."
3,2221066352,1583419016,1.583556e+09,NaN,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number..."
4,2222478803,1583318305,1.583480e+09,NaN,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...


In [7]:
## SLA matrix
matrix = df_sla.iloc[1:5, 2:].values
city_order = ["metro manila", "luzon", "visayas", "mindanao"]

In [8]:
matrix

array([['3 working days', '5 working days', '7 working days',
        '7 working days'],
       ['5 working days', '5 working days', '7 working days',
        '7 working days'],
       ['7 working days', '7 working days', '7 working days',
        '7 working days'],
       ['7 working days', '7 working days', '7 working days',
        '7 working days']], dtype=object)

In [9]:
## SLA 2nd attempt
print(df_sla.iloc[8:,:1].values[0][0])

2nd Attempt must be no later than 3 working days after the 1st Attempt, regardless of origin to destination route (Day 0 = Day of 1st Attempt; Day 1 = Next Day after 1st Attempt).


#### Create sample df

In [10]:
## Sample orders
df_sample = df_delivery.sample(n=1000)
df_sample.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
2676327,2306750619,1584053169,1.584177e+09,1.584524e+09,LHINITTE Audit-DPWH Ramtoda 130B-Pagasa Eber (...,"Zapote, Village,Eusebio (get Cloths, Rm210 par..."
3057979,2313270732,1584603369,1.584949e+09,NaN,"Brgy.Suba,Purok 1,Phase 10-s QUIOT, 0458,buton...","castaneda Ave.Ext. CPT ave,odeon serrano. Blk3..."
32792,2234158722,1583578260,1.583830e+09,NaN,"bandung St.,cor.M.paterno L-24, Tell 2943-G st...","C11,Saulog 2d08 paz S-18C FILINVEST 1, PB001 G..."
2453474,2303291547,1584088908,1.584676e+09,1.585015e+09,baboyan F.V.R. Rosalina's 21F-Pasco Astrera re...,Peta Partnership tamarind Intalan VASRA Valenc...
1790981,2342629171,1584778922,1.586149e+09,NaN,"St.,Aplaya,Bauan,Batangas Rizalino ZAKRAH dang...","9025 Duyan-Duyan, Bldg,M WEST 1533 Avenue,Pasa..."


#### Find origin, destination, SLA

In [11]:
def find_city(x):
    for city in city_order:
        if city.lower() in x:
            return city.lower()

In [12]:
## Get Origin
def get_origin(df):
    origin = find_city(df['selleraddress'].lower())
    return origin

In [13]:
## Get Destination
def get_destination(df):
    destination = find_city(df['buyeraddress'].lower())
    return destination

In [14]:
df_sample['origin'] = df_sample.apply(get_origin, axis=1)
df_sample['destination'] = df_sample.apply(get_destination, axis=1)

In [15]:
df_sample.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,origin,destination
2676327,2306750619,1584053169,1.584177e+09,1.584524e+09,LHINITTE Audit-DPWH Ramtoda 130B-Pagasa Eber (...,"Zapote, Village,Eusebio (get Cloths, Rm210 par...",metro manila,metro manila
3057979,2313270732,1584603369,1.584949e+09,NaN,"Brgy.Suba,Purok 1,Phase 10-s QUIOT, 0458,buton...","castaneda Ave.Ext. CPT ave,odeon serrano. Blk3...",metro manila,luzon
32792,2234158722,1583578260,1.583830e+09,NaN,"bandung St.,cor.M.paterno L-24, Tell 2943-G st...","C11,Saulog 2d08 paz S-18C FILINVEST 1, PB001 G...",metro manila,metro manila
2453474,2303291547,1584088908,1.584676e+09,1.585015e+09,baboyan F.V.R. Rosalina's 21F-Pasco Astrera re...,Peta Partnership tamarind Intalan VASRA Valenc...,metro manila,visayas
1790981,2342629171,1584778922,1.586149e+09,NaN,"St.,Aplaya,Bauan,Batangas Rizalino ZAKRAH dang...","9025 Duyan-Duyan, Bldg,M WEST 1533 Avenue,Pasa...",metro manila,visayas


In [16]:
def get_sla(df):
    sla = matrix[city_order.index(df['origin']), city_order.index(df['destination'])]
    days = int(re.search(r'\d', sla).group(0))
    return days

In [17]:
df_sample['sla'] = df_sample.apply(get_sla, axis=1)

In [18]:
df_sample.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,origin,destination,sla
2676327,2306750619,1584053169,1.584177e+09,1.584524e+09,LHINITTE Audit-DPWH Ramtoda 130B-Pagasa Eber (...,"Zapote, Village,Eusebio (get Cloths, Rm210 par...",metro manila,metro manila,3
3057979,2313270732,1584603369,1.584949e+09,NaN,"Brgy.Suba,Purok 1,Phase 10-s QUIOT, 0458,buton...","castaneda Ave.Ext. CPT ave,odeon serrano. Blk3...",metro manila,luzon,5
32792,2234158722,1583578260,1.583830e+09,NaN,"bandung St.,cor.M.paterno L-24, Tell 2943-G st...","C11,Saulog 2d08 paz S-18C FILINVEST 1, PB001 G...",metro manila,metro manila,3
2453474,2303291547,1584088908,1.584676e+09,1.585015e+09,baboyan F.V.R. Rosalina's 21F-Pasco Astrera re...,Peta Partnership tamarind Intalan VASRA Valenc...,metro manila,visayas,7
1790981,2342629171,1584778922,1.586149e+09,NaN,"St.,Aplaya,Bauan,Batangas Rizalino ZAKRAH dang...","9025 Duyan-Duyan, Bldg,M WEST 1533 Avenue,Pasa...",metro manila,visayas,7


#### Check for first SLA fullfilment

In [ ]:
## Convert all date column to date
pick = pd.to_datetime(df_sample['pick'], unit='s').dt.date
first_deliver = pd.to_datetime(df_sample['1st_deliver_attempt'], unit='s').dt.date
second_deliver = pd.to_datetime(df_sample['2nd_deliver_attempt'], unit='s').dt.date

In [ ]:
df_sample['pick'] = pick
df_sample['1st_deliver_attempt'] = first_deliver
df_sample['2nd_deliver_attempt'] = second_deliver

In [ ]:
df_sample.head()

In [ ]:
## Initiate public holidays
public_holidays = ["2020-03-08", "2020-03-25", "2020-03-30", "2020-03-31"]

In [ ]:
def get_busday_first(df):
    create_date = str(df['pick'])
    resolve_date = str(df['1st_deliver_attempt'])

    create_datetime = datetime.strptime(create_date, '%Y-%m-%d')
    resolve_datetime = datetime.strptime(resolve_date, '%Y-%m-%d')

    busday = np.busday_count(create_date, resolve_date, holidays=public_holidays, weekmask=[1,1,1,1,1,1,0])

    return busday    

In [ ]:
df_sample['1st_deliver_days'] = df_sample.apply(get_busday_first, axis=1)

In [ ]:
df_sample.head()

In [ ]:
## Saving checkpoint
# import pickle
# pickle.dump(df_sample, open("1st_fullfilment.pickle", "wb"))

#### Check for 2nd SLA fullfilment

In [ ]:
def get_busday_second(df):
    create_date = str(df['1st_deliver_attempt'])
    resolve_date = str(df['2nd_deliver_attempt'])
    
    if resolve_date == 'NaT':
        resolve_date = create_date

    create_datetime = datetime.strptime(create_date, '%Y-%m-%d')
    resolve_datetime = datetime.strptime(resolve_date, '%Y-%m-%d')

    busday = np.busday_count(create_date, resolve_date, holidays=public_holidays, weekmask=[1,1,1,1,1,1,0])

    return busday   

In [ ]:
df_sample['2nd_deliver_days'] = df_sample.apply(get_busday_second, axis=1)

In [ ]:
df_sample.head()

In [ ]:
## Saving checkpoint
# import pickle
# pickle.dump(df_sample, open("2nd_fullfilment.pickle", "wb"))

#### Decide late/not late

In [ ]:
not_late_df = df_sample[(df_sample['1st_deliver_days'] <= df_sample['sla']) & (df_sample['2nd_deliver_days'] <= 3)][['orderid']]
not_late_df['is_late'] = 0
not_late_df.head()

In [ ]:
late_df = df_sample[~df_sample['orderid'].isin(not_late_df['orderid'])][['orderid']]
late_df['is_late'] = 1
late_df.head()

In [ ]:
not_late_df.orderid.nunique() + late_df.orderid.nunique()

In [ ]:
res_df = pd.concat([late_df, not_late_df], axis=0)
res_df.head()

### Run all below cells as "Code" to export the solution for sample/all data

#### Export solution
res_df.to_csv("solution_logistics_bahy_sample.csv", index=False)

test = pd.read_csv("solution_logistics_bahy_sample.csv")
test.head()

test.shape

#### Test to all data
Just replace all the cells below to a Code cell to run it on all data

df_delivery.head()

df_delivery['origin'] = df_delivery.apply(get_origin, axis=1)
df_delivery['destination'] = df_delivery.apply(get_destination, axis=1)

df_delivery['sla'] = df_delivery.apply(get_sla, axis=1)

pick = pd.to_datetime(df_delivery['pick'], unit='s').dt.date
first_deliver = pd.to_datetime(df_delivery['1st_deliver_attempt'], unit='s').dt.date
second_deliver = pd.to_datetime(df_delivery['2nd_deliver_attempt'], unit='s').dt.date

df_delivery['pick'] = pick
df_delivery['1st_deliver_attempt'] = first_deliver
df_delivery['2nd_deliver_attempt'] = second_deliver

public_holidays = ["2020-03-08", "2020-03-25", "2020-03-30", "2020-03-31"]

df_delivery['1st_deliver_days'] = df_delivery.apply(get_busday_first, axis=1)

df_delivery['2nd_deliver_days'] = df_delivery.apply(get_busday_second, axis=1)

not_late_df = df_sample[(df_sample['1st_deliver_days'] <= df_sample['sla']) & (df_sample['2nd_deliver_days'] <= 3)][['orderid']]
not_late_df['is_late'] = 0

late_df = df_sample[~df_sample['orderid'].isin(not_late_df['orderid'])][['orderid']]
late_df['is_late'] = 1

res_df = pd.concat([late_df, not_late_df], axis=0)

#### Export solution
res_df.to_csv("solution_logistics_bahy.csv", index=False)